In [1]:
import pandas as pd, numpy as np
from db_queries import get_population
!date
!whoami

Sat Mar 27 01:39:19 PDT 2021
ndbs


# File where GBD data is stored

In [2]:
share_directory = '/share/scratch/users/ndbs/vivarium_lsff/gbd_data'
hdfstore_path = f'{share_directory}/multmodel_data.hdf'

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Read in folic acid DALYs averted, compute averted rates and PIF, and save results

**Notes:**
* The filepath for folic acid DALYs averted needs to be updated when new data is available.
* Folic acid data only has years 2022-2025, whereas other nutrients have years 2021-2025.
* This code was copied from a version of Ali's notebook `location_specific_results.ipynb`, and modified slightly.

In [3]:
folic_acid_dalys = pd.read_csv('folic_acid/tier1_2_3_dalys_averted_folic_acid_03_25_2021.csv')
folic_acid_dalys['measure'] = 'counts_averted'

folic_acid_location_ids = list(folic_acid_dalys.location_id.unique())

pop = get_population(location_id=folic_acid_location_ids,
                     year_id=2019,
                     sex_id=[1,2],
                     age_group_id=[2,3,4,5],
                     gbd_round_id=6,
                     decomp_step='step4').drop(columns='year_id').groupby('location_id', as_index=False).sum()
folic_acid_daly_rate = folic_acid_dalys.merge(pop, on='location_id')
for i in list(range(0,1000)):
    folic_acid_daly_rate[f'draw_{i}'] = (folic_acid_daly_rate[f'draw_{i}'] 
     / folic_acid_daly_rate['population']
     * 100_000)
folic_acid_daly_rate['measure'] = 'rates_averted'
folic_acid_daly_rate = folic_acid_daly_rate.set_index(['location_id','year_id','coverage_level','measure'])
folic_acid_daly_rate = (folic_acid_daly_rate
                        .drop(columns=[c for c in folic_acid_daly_rate.columns if 'draw' not in c])
                        .reset_index())
folic_acid = pd.concat([folic_acid_dalys, folic_acid_daly_rate], ignore_index=True)

# folic_acid_total_dalys = pull_dalys([642], [642], list(folic_acid_dalys.location_id.unique()), 
#                               [2,3,4,5], [1,2], ['location_id','sex_id','age_group_id'])
folic_acid_total_dalys = pd.read_hdf(hdfstore_path, key='/neural_tube_defects/dalys_bmgf_25_countries')
folic_acid_total_dalys = folic_acid_total_dalys.query("location_id in @folic_acid_location_ids")
folic_acid_total_dalys = folic_acid_total_dalys.groupby(['location_id']).sum()

folic_acid_pif = (folic_acid_dalys.set_index(['location_id','year_id','coverage_level','measure']) 
                  / folic_acid_total_dalys * 100).reset_index()
folic_acid_pif['measure'] = 'pif'
folic_acid = pd.concat([folic_acid, folic_acid_pif], ignore_index=True)
# folic_acid['nutrient'] = 'folic_acid'
folic_acid = folic_acid.rename(columns={'year_id':'year'})
# Save folic acid data in same format as other nutrients
folic_acid.to_pickle('results_raw/folic_acid.pkl')

# # Save folic acid data in same format as other nutrients
# folic_acid.drop(columns='nutrient').to_pickle('results_raw/folic_acid.pkl')

folic_acid

,location_id,year,coverage_level,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,...,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,measure
0,15,2022,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,counts_averted
1,15,2022,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,counts_averted
2,15,2022,0.8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,counts_averted
3,15,2023,0.2,103.130897,188.817546,120.000370,126.277878,37.671687,83.550520,91.681835,...,84.670059,67.536953,76.250901,69.020125,64.092056,94.783455,155.022355,126.734799,133.221816,counts_averted
4,15,2023,0.5,257.827242,472.043865,300.000926,315.694694,94.179218,208.876299,229.204589,...,211.675146,168.842384,190.627254,172.550312,160.230139,236.958637,387.555888,316.836998,333.054539,counts_averted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,214,2024,0.5,10.681798,8.158414,8.525188,9.115311,7.732866,8.645058,8.598805,...,6.307944,7.505265,8.743295,5.987141,6.962028,7.252745,6.339637,6.302917,6.194036,pif
464,214,2024,0.8,17.090876,13.053462,13.640301,14.584497,12.372586,13.832092,13.758088,...,10.092711,12.008424,13.989272,9.579426,11.139244,11.604391,10.143419,10.084667,9.910457,pif
465,214,2025,0.2,4.332142,3.306500,3.472559,3.703594,3.136036,3.509733,3.494097,...,2.669522,3.202188,3.662139,2.541183,2.928038,3.072962,2.684035,2.657365,2.614646,pif
466,214,2025,0.5,10.830356,8.266251,8.681397,9.258985,7.840091,8.774333,8.735243,...,6.673805,8.005469,9.155347,6.352958,7.320094,7.682405,6.710086,6.643412,6.536615,pif


# Read in locations and results for all nutrients, and concatenate

**Notes:**
* Currently this only works with a single vehicle for each nutrient. A better strategy in the future would be to save the vehicle name in the dataframe when the results are generated, rather than adding it here. The data for all vehicles could be concatenated before saving so that only one file for each nutrient needs to be read in here.

In [4]:
locations = pd.read_csv('../gbd_data_summary/input_data/bmgf_top_25_countries_20201203.csv')

folic_acid = pd.read_pickle('results_raw/folic_acid.pkl')
folic_acid['nutrient'] = 'Folic acid'
folic_acid['vehicle'] = 'Wheat flour'

zinc = pd.read_pickle('results_raw/zinc.pkl')
zinc['nutrient'] = 'Zinc'
zinc['vehicle'] = 'Wheat flour'

vitamin_a = pd.read_pickle('results_raw/vitamin_a.pkl')
vitamin_a['nutrient'] = 'Vitamin A'
vitamin_a['vehicle'] = 'Oil'

iron_u5 = pd.read_pickle('results_raw/iron_u5.pkl')
iron_u5['nutrient'] = 'Iron'
iron_u5['vehicle'] = 'Wheat flour'

iron_wra = pd.read_pickle('results_raw/iron_wra.pkl')
iron_wra['nutrient'] = 'Iron'
iron_wra['vehicle'] = 'Wheat flour'

iron = (iron_wra.set_index([c for c in iron_wra.columns if 'draw' not in c]) 
        + iron_u5.set_index([c for c in iron_u5.columns if 'draw' not in c])).reset_index()

results = pd.concat([zinc, vitamin_a, iron, folic_acid], ignore_index=True).merge(locations, on='location_id')
results

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,location_id,year,coverage_level,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,...,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,measure,nutrient,vehicle,location_name
0,15,2021,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pif,Zinc,Wheat flour,Myanmar
1,15,2021,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pif,Zinc,Wheat flour,Myanmar
2,15,2021,0.8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pif,Zinc,Wheat flour,Myanmar
3,15,2022,0.2,0.209826,0.462305,0.470934,0.426342,0.403176,0.405280,0.350347,...,0.382032,0.393948,0.327077,0.451717,0.471465,0.478722,pif,Zinc,Wheat flour,Myanmar
4,15,2022,0.5,0.524565,1.155762,1.177334,1.065854,1.007940,1.013200,0.875867,...,0.955081,0.984870,0.817694,1.129293,1.178663,1.196805,pif,Zinc,Wheat flour,Myanmar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2218,214,2024,0.5,10.681798,8.158414,7.979377,7.359153,8.946206,8.926295,6.345747,...,5.987141,6.962028,7.252745,6.339637,6.302917,6.194036,pif,Folic acid,Wheat flour,Nigeria
2219,214,2024,0.8,17.090876,13.053462,12.767004,11.774644,14.313929,14.282073,10.153195,...,9.579426,11.139244,11.604391,10.143419,10.084667,9.910457,pif,Folic acid,Wheat flour,Nigeria
2220,214,2025,0.2,4.332142,3.306500,3.236747,3.008976,3.646911,3.640727,2.606401,...,2.541183,2.928038,3.072962,2.684035,2.657365,2.614646,pif,Folic acid,Wheat flour,Nigeria
2221,214,2025,0.5,10.830356,8.266251,8.091867,7.522439,9.117278,9.101817,6.516003,...,6.352958,7.320094,7.682405,6.710086,6.643412,6.536615,pif,Folic acid,Wheat flour,Nigeria


# Write function to process results into desired format

## Desired columns:

location, coverage_level, vehicle, nutrient, measure (counts averted, rates averted, pifs), value, lower, upper

In [5]:
# location, vehicle, nutrient, measure (counts averted, rates averted, pifs), value, lower, upper
print(results.shape)

def process_results(results):
    processed = (
        results
        .set_index([c for c in results.columns if 'draw' not in c])
        .xs(2025, level='year')
        .T.describe(percentiles=[0.025, 0.975]).T # For some reason, using .agg instead drops index level names. Ugh.
        .rename(columns={'mean':'value', '2.5%':'lower', '97.5%':'upper'})
        .reset_index()
        [['location_name', 'nutrient', 'vehicle', 'coverage_level', 'measure', 'value', 'lower', 'upper']]
        .assign(measure_order=lambda df: df.measure.map({'counts_averted':1, 'rates_averted':2, 'pif':3}))
        .sort_values(['location_name', 'nutrient', 'vehicle', 'measure_order', 'coverage_level'])
        .drop(columns='measure_order')
        .replace(['counts_averted', 'rates_averted', 'pif'],
                 ['DALYs averted', 'DALYs averted per 100,000 PYs', 'Potential Impact Fraction (%)'])
    )
    return processed

processed = process_results(results)
processed
# location, vehicle, nutrient, measure (counts averted, rates averted, pifs), value, lower, upper

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2223, 1007)


,location_name,nutrient,vehicle,coverage_level,measure,value,lower,upper
99,Bangladesh,Folic acid,Wheat flour,0.2,DALYs averted,3051.474510,1009.946923,6873.203470
100,Bangladesh,Folic acid,Wheat flour,0.5,DALYs averted,7628.686275,2524.867308,17183.008674
101,Bangladesh,Folic acid,Wheat flour,0.8,DALYs averted,12205.898040,4039.787693,27492.813879
102,Bangladesh,Folic acid,Wheat flour,0.2,"DALYs averted per 100,000 PYs",22.199142,7.347253,50.001801
103,Bangladesh,Folic acid,Wheat flour,0.5,"DALYs averted per 100,000 PYs",55.497856,18.368133,125.004503
...,...,...,...,...,...,...,...,...
43,Viet Nam,Zinc,Wheat flour,0.5,"DALYs averted per 100,000 PYs",0.078209,0.000348,0.276742
44,Viet Nam,Zinc,Wheat flour,0.8,"DALYs averted per 100,000 PYs",0.125135,0.000557,0.442788
36,Viet Nam,Zinc,Wheat flour,0.2,Potential Impact Fraction (%),1.921092,0.729982,3.382712
37,Viet Nam,Zinc,Wheat flour,0.5,Potential Impact Fraction (%),4.802731,1.824956,8.456779


# Save `.csv` of processed results and read it to check format

In [6]:
filename = 'results_tables/2021_03_26_results_tier_1_2_3.csv'
processed.to_csv('results_tables/2021_03_26_results_tier_1_2_3.csv', index=False)
pd.read_csv(filename)

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,location_name,nutrient,vehicle,coverage_level,measure,value,lower,upper
0,Bangladesh,Folic acid,Wheat flour,0.2,DALYs averted,3051.474510,1009.946923,6873.203470
1,Bangladesh,Folic acid,Wheat flour,0.5,DALYs averted,7628.686275,2524.867308,17183.008674
2,Bangladesh,Folic acid,Wheat flour,0.8,DALYs averted,12205.898040,4039.787693,27492.813879
3,Bangladesh,Folic acid,Wheat flour,0.2,"DALYs averted per 100,000 PYs",22.199142,7.347253,50.001801
4,Bangladesh,Folic acid,Wheat flour,0.5,"DALYs averted per 100,000 PYs",55.497856,18.368133,125.004503
...,...,...,...,...,...,...,...,...
463,Viet Nam,Zinc,Wheat flour,0.5,"DALYs averted per 100,000 PYs",0.078209,0.000348,0.276742
464,Viet Nam,Zinc,Wheat flour,0.8,"DALYs averted per 100,000 PYs",0.125135,0.000557,0.442788
465,Viet Nam,Zinc,Wheat flour,0.2,Potential Impact Fraction (%),1.921092,0.729982,3.382712
466,Viet Nam,Zinc,Wheat flour,0.5,Potential Impact Fraction (%),4.802731,1.824956,8.456779
